<a href="https://colab.research.google.com/github/Melanee-Melanee/Old-Persian-Cuneiform-OCR/blob/main/easyocr_old_persian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/JaidedAI/EasyOCR.git

fatal: destination path 'EasyOCR' already exists and is not an empty directory.


In [3]:
!pip install easyocr


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 21.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

In [17]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.15.0
backcall                         0.2.0
beautifulsoup4                   4.12.3

In [2]:
%cd /content/EasyOCR/trainer/

/content/EasyOCR/trainer


In [3]:
!pwd

/content/EasyOCR/trainer


In [10]:
!unzip /content/EasyOCR/trainer/all_data/en_train_filtered.zip

Archive:  /content/EasyOCR/trainer/all_data/en_train_filtered.zip
   creating: en_train_filtered/
  inflating: en_train_filtered/Darayavaus.png  
  inflating: en_train_filtered/line1.png  
  inflating: en_train_filtered/thatiy .png  
  inflating: en_train_filtered/xsaya.png  
  inflating: en_train_filtered/Auramazda.png  
  inflating: en_train_filtered/baratuv.png  
  inflating: en_train_filtered/line2.png  
  inflating: en_train_filtered/mana.png  
  inflating: en_train_filtered/upastam.png  
  inflating: en_train_filtered/bagaibis.png  
  inflating: en_train_filtered/hada.png  
  inflating: en_train_filtered/line3.png  
  inflating: en_train_filtered/uta.png  
  inflating: en_train_filtered/vithaibis.png  
  inflating: en_train_filtered/dahyaum .png  
  inflating: en_train_filtered/imam.png  
  inflating: en_train_filtered/line4.png  
  inflating: en_train_filtered/haca.png  
  inflating: en_train_filtered/haca2.png  
  inflating: en_train_filtered/hainaya.png  
  inflating: en_train

In [11]:
!unzip /content/EasyOCR/trainer/all_data/en_val.zip

Archive:  /content/EasyOCR/trainer/all_data/en_val.zip
   creating: en_val/
  inflating: en_val/Auramzda2.png    
  inflating: en_val/line7.png        
  inflating: en_val/haca3.png        
  inflating: en_val/imam2.png        
  inflating: en_val/aita2.png        
  inflating: en_val/labels.csv       


In [4]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [5]:
cudnn.benchmark = True
cudnn.deterministic = False

In [6]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [7]:
opt = get_config("config_files/en_filtered_config.yaml")
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data
opt.select_data: ['en_train_filtered']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data	 dataset: en_train_filtered
all_data/en_train_filtered
sub-directory:	/en_train_filtered	 num samples: 34
num total samples of en_train_filtered: 34 x 1.0 (total_data_usage_ratio) = 34
num samples of en_train_filtered per batch: 32 x 1.0 (batch_ratio) = 32


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


--------------------------------------------------------------------------------
Total_batch_size: 32 = 32
--------------------------------------------------------------------------------
dataset_root:    all_data/en_val	 dataset: /
all_data/en_val/
sub-directory:	/.	 num samples: 4
--------------------------------------------------------------------------------
No Transformation module specified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


model input parameters 64 600 20 1 256 256 97 34 None VGG BiLSTM CTC
Model:
DataParallel(
  (module): Model(
    (FeatureExtraction): VGG_FeatureExtractor(
      (ConvNet): Sequential(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): ReLU(inplace=True)
        (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (7): ReLU(inplace=True)
        (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (9): ReLU(inplace=True)
        (10): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
        (11): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


Exception ignored in: <function _after_fork at 0x7b2ce5319990>
Traceback (most recent call last):
Exception ignored in:   File "/usr/lib/python3.10/threading.py", line 1635, in _after_fork
<function _after_fork at 0x7b2ce5319990>    thread._reset_internal_locks(False)

  File "/usr/lib/python3.10/threading.py", line 885, in _reset_internal_locks
Traceback (most recent call last):
    def _reset_internal_locks(self, is_alive):  File "/usr/lib/python3.10/threading.py", line 1635, in _after_fork

    KeyboardInterruptthread._reset_internal_locks(False): 

KeyboardInterrupt: 


KeyboardInterrupt: 

In [11]:
!cp /root/EasyOCR//user_network/custom_example.yaml /root/.EasyOCR//user_network/custom_example.yaml



In [12]:
!cp /root/EasyOCR/user_network/custom_example.py /root/.EasyOCR/user_network/custom_example.py

In [8]:
!cp /root/EasyOCR/model/custom_example.pth /root/.EasyOCR/model/custom_example.pth

In [9]:
#single word:
import easyocr
reader = easyocr.Reader(['op'], recog_network='custom_example')
result = reader.readtext('/content/Auramazda.png', detail=0)
print(result)

['da :']


In [10]:
#single word:
import easyocr
reader = easyocr.Reader(['op'], recog_network='custom_example')
result = reader.readtext('/content/drauga.png', detail=0)
print(result)

['dahya :']


In [14]:
#single word:
import easyocr
reader = easyocr.Reader(['op'], recog_network='custom_example')
result = reader.readtext('/content/imam2.png', detail=0)
print(result)

['dram :']


In [15]:
#single lines:
reader = easyocr.Reader(['op'], recog_network='custom_example')
result = reader.readtext('/content/line7.png', detail=0)
print(result)

['dram :', 'dam :', 'ma :', 'dra :', 'daham :']


In [16]:
#multiple lines
text = ''
results = reader.readtext('/content/DPd.png')
for result in results:
  text = text + result[1] +  ' '

text = text[:-1]
print(text)

dahaiy : aitavabis : dayamiy : ma : a : daya : jayatuv : haya : Darayabis  pabis : daya : aitaum : da : daha : datuv : xsa : sa : hainya : ha : hainya : da : aiy : aita : ba : dam : a : daya : dam : dahyam : ba: dam  dadyaiy : aitaiy : datyatuv :
